In [12]:
import os
import gensim
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import pickle
from random import shuffle



stop_word_english = set(stopwords.words("english"))
#elements are tuples in which the first element is the toknized words and the second is the label
# 0 is label positive and 1 is negative
all_txt = []
num_features = 150


In [27]:
def remove_stop_words(document):
    for w in document:
        if w in stop_word_english:
            document.remove(w)

In [15]:
for filename in os.listdir("/home/sufon/nltk_data/corpora/movie_reviews/pos/"):
    with open("/home/sufon/nltk_data/corpora/movie_reviews/pos/"+filename, 'r') as myfile:
        data=gensim.utils.simple_preprocess(myfile.read().replace('\n', ''))
        all_txt.append((data, 0))

In [16]:
for filename in os.listdir("/home/sufon/nltk_data/corpora/movie_reviews/neg/"):
    with open("/home/sufon/nltk_data/corpora/movie_reviews/neg/"+filename, 'r') as myfile:
        data=gensim.utils.simple_preprocess(myfile.read().replace('\n', ''))
        all_txt.append((data,1))

In [17]:
for review in all_txt:
    remove_stop_words(review[0])

In [18]:
txts = [txt[0] for txt in all_txt]

In [15]:
model = gensim.models.Word2Vec(
        txts,
        size=150,
        window=10,
        min_count=5,
        workers=10,iter=10,sample=0.01)

In [16]:
model.train(txts, total_examples=len(all_txt), epochs=10)

(7996748, 8560090)

In [38]:
vocabulary = set(model.wv.index2word)

In [18]:
d = {}
index = 0

for doc in all_txt:
    txt = doc[0]
    lst = []
    for vocab_word in list(model.wv.vocab.keys()):
        if vocab_word in txt:
            lst.append(model[vocab_word])
        else:
            #check if it is right to add a 0 here
            lst.append(0)
            
    lst.append(doc[1])
    d[index] = lst
    
    index = index + 1    

/home/sufon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [19]:
cols = list((model.wv.vocab.keys()))
cols.append("label")
df = pd.DataFrame.from_dict(d, orient="index",columns=cols)

In [26]:
#with open('movie_rating_word2vec_dataframe.pkl', 'wb') as output:
    #pickle.dump(df, output)
    
#with open('word2vec_trained_model.pkl', 'wb') as output:
    #pickle.dump(model, output)

In [33]:
#READ PICKLE
#pickle_in = open('movie_rating_word2vec_dataframe.pkl','rb')
#df = pickle.load(pickle_in)

#pickle_in_word2vec = open('word2vec_trained_model.pkl','rb')
#model = pickle.load(pickle_in_word2vec)

In [34]:
#returns the average, 150 dimensioned, vector embedding of every word in a given document
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words[0]:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

In [35]:
#get the feature vector of each document in the corpus, and return them as an np array
def averaged_word_vectorizer(corpus, model, num_features):
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [36]:
w2v_feature_array = averaged_word_vectorizer(corpus=all_txt#all_txt here is a tuple of (words, label)
                                             , model=model,
                                             num_features=150)

In [25]:
length_of_feature_array = len(w2v_feature_array)
data = []
for i in range (length_of_feature_array):
    data.append( (w2v_feature_array[i], all_txt[i][1]) )

In [26]:
shuffle(data)

In [28]:
#with open('data_pairs.pkl', 'wb') as output:
    #pickle.dump(data, output)

In [13]:
#pickle_in = open('data_pairs.pkl','rb')
#data = pickle.load(pickle_in)

In [14]:
x = []
y = []

for data_pair in data[:]:
    x.append(data_pair[0])
    y.append(data_pair[1])

# here we use the SVM aproach

In [62]:
from sklearn import svm

clf = svm.SVC(gamma=0.001,C=100)

In [63]:
clf.fit(x[:1700],y[:1700])

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [64]:
print(clf.score(x[1700:],y[1700:]))

0.8133333333333334


# here we use CNN aproach

In [48]:
for data_pair in data:
    data_pair[0].reshape(150,1,1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model_cnn = Sequential()
#add model layers
model_cnn.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(150,1,1)))
model_cnn.add(Conv2D(32, kernel_size=3, activation="relu"))
model_cnn.add(Flatten())
model_cnn.add(Dense(10, activation="softmax"))

In [ ]:
#compile model using accuracy to measure model performance
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# here we use the Neural Network aproach

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [5]:
# load dataset
dataframe = pd.read_csv("/home/sufon/iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [21]:
print(X.shape)

(150, 4)


In [16]:
one_hot_labels = np_utils.to_categorical(y)

In [17]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=150, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [18]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [24]:
x = np.array(x)

In [51]:
estimator.fit(x[:500], one_hot_labels[:500])

In [52]:
predictions = estimator.predict(x[500:1000])

In [50]:
with open("/home/sufon/Documents/test", 'r') as myfile:
    test_data = gensim.utils.simple_preprocess(myfile.read().replace('\n', ''))

remove_stop_words(test_data)
test_data_feature_vector = [average_word_vectors(test_data, model, vocabulary, num_features)]

AttributeError: 'KerasClassifier' object has no attribute 'model'